In [15]:
import pandas as pd
import re
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

train = pd.read_csv('train.csv')
print("Training Set:"% train.columns, train.shape, len(train))
test = pd.read_csv('test.csv')
print("Test Set:"% test.columns, test.shape, len(test))
def  clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower() #lower used to make all letter become lower care
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    #re.sub(a,b,c) is used to substitute "a" to "b" from "c"
    return df
test_clean = clean_text(test, "tweet")
train_clean = clean_text(train, "tweet")
#upsampling the data because hatespeech data is low, hatespeech = 1 while not hate = 0
train_majority = train_clean[train_clean.label==0]
train_minority = train_clean[train_clean.label==1]
train_minority_upsampled = resample(train_minority, 
                                 replace=True,    
                                 n_samples=len(train_majority),   
                                 random_state=123)
train_upsampled = pd.concat([train_minority_upsampled, train_majority])
train_upsampled['label'].value_counts()
#making model pipeline, pipeline is like workflow of the model to process the data
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', SGDClassifier()),])
X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'],
                                                   train_upsampled['label'],random_state = 0)


model = pipeline_sgd.fit(X_train, y_train)
y_predict = model.predict(X_test)
f1_score(y_test, y_predict)

#for test_clean is not labeled, but can be used for unsupervised learning which will take another way, not in this program.

Training Set: (31962, 3) 31962
Test Set: (17197, 2) 17197


0.9696